In [1]:
import subprocess
import pandas as pd
import numpy as np
import json
from timeit import default_timer
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp
import os
import tqdm

sns.set()

In [2]:
subprocess.check_output

<function subprocess.check_output(*popenargs, timeout=None, **kwargs)>

In [3]:
df = pd.read_csv('data/sample_full_text.csv')

df.head()

,question,document,type,order,text,pmid
0,1061,0,1_title,0,Hirschsprung associated GDNF mutations do not ...,11973622
1,1061,0,2_abstract_sentence,0,Hirschsprung disease (HSCR) is a complex disor...,11973622
2,1061,0,2_abstract_sentence,1,The highest proportion of both familial and sp...,11973622
3,1061,0,2_abstract_sentence,2,Five germline mutations in the glial cell-line...,11973622
4,1061,0,2_abstract_sentence,3,Pedigrees analysis and the observed associati...,11973622


In [4]:
# with open('data/tmp/test_1.txt', 'w') as file:
#     for l in df['text'].tolist():
#         file.write(l)
#         file.write('\n')

In [5]:
# !head data/tmp/test_1.txt

In [6]:
# process = subprocess.run(["python", 
#                          "bert_resources/bert/extract_features.py", 
#                          "--input_file=data/tmp/test_1.txt",
#                          "--output_file=data/embeddings/test_1.jsonl",
#                          "--vocab_file=bert_resources/uncased_L-2_H-128_A-2/vocab.txt",
#                          "--bert_config_file=bert_resources/uncased_L-2_H-128_A-2/bert_config.json",
#                          "--init_checkpoint=bert_resources/uncased_L-2_H-128_A-2/bert_model.ckpt",
#                          "--layers=-1",
#                          "--max_seq_length=128",
#                          "--batch_size=8"])

In [7]:
# process.returncode

In [8]:
# with open('data/embeddings/test_1.jsonl', 'r') as file:
#     string = json.loads(file.readline())

# string['features'][0]['layers'][0]['values']

In [9]:
def extract_cls_embedding(text_list, 
                          tmp_file_name = 'tmp', 
                          tmp_folder = 'tmp', 
                          data_folder = 'data/testing',
                          output_folder = 'json',
                          bert_folder = 'bert_resources/bert_models/uncased_L-2_H-128_A-2'):     
        
    with open('{}/{}/{}.txt'.format(data_folder, tmp_folder, tmp_file_name), 'w') as file:
#         for l in text_list:
#             file.write(l)
#             file.write('\n')
        file.write('\n'.join(text_list))
             
    process = subprocess.run(["python", 
                               "bert_resources/bert/extract_features.py", 
                               "--input_file={}/{}/{}.txt".format(data_folder, tmp_folder, tmp_file_name),
                               "--output_file={}/{}/{}.jsonl".format(data_folder, output_folder, tmp_file_name),
                               "--vocab_file={}/vocab.txt".format(bert_folder),
                               "--bert_config_file={}/bert_config.json".format(bert_folder),
                               "--init_checkpoint={}/bert_model.ckpt".format(bert_folder),
                               "--layers=-1",
                               "--max_seq_length=128",
                               "--batch_size=8"])
    if process.returncode != 0:
        raise ValueError('There was a problem with the subprocess')
        
    with open('{}/{}/{}.jsonl'.format(data_folder, output_folder, tmp_file_name), 'r') as file:
        jsons = [json.loads(l) for l in file.readlines()]
        
    embeddings = [np.array(j['features'][0]['layers'][0]['values']) for j in jsons]
    
    return embeddings

In [10]:
# data = []
# for i in range(10, 100):
#     start = default_timer()
#     s = extract_cls_embedding(df['text'][:i].tolist())
#     print('{}, {:.2f}'.format(i, default_timer() - start))
#     data.append(default_timer() - start)
    
# plt.plot(data)

In [11]:
# start = default_timer()
# s_1 = extract_cls_embedding(df['text'].tolist())
# print(default_timer() - start)
# len(s_1)

In [12]:
# start = default_timer()
# s_2 = extract_cls_embedding(df['text'].tolist(), bert_folder='bert_resources/uncased_L-12_H-768_A-12')
# print(default_timer() - start)
# len(s_1)

In [13]:
# full_text = pd.read_csv('data/raw/full_text.csv')
# full_text.shape

In [14]:
def produce_batches(data, batch_size):
    batches = []
    for i in range(len(data) // batch_size + 1):
        batches.append(data[i*batch_size:(i + 1)*batch_size])
    #import pdb; pdb.set_trace()
    output = [('batch_{}'.format(str(i).rjust(int(np.ceil(np.log10(len(data) // batch_size))),'0')), b) for i, b in enumerate(batches)]
    return output

In [15]:
batches = produce_batches(df['text'].tolist(), 800)

[b for b,l in batches]

['batch_0', 'batch_1', 'batch_2']

In [16]:
def extract_emb(x):
    return (x[0], extract_cls_embedding(x[1], tmp_file_name = x[0]))

In [17]:
start = default_timer()

results_1 = extract_cls_embedding(df['text'].tolist())

default_timer() - start

50.94143890000123

In [18]:
# results_1[0]

In [19]:
p = mp.Pool(processes = os.cpu_count() - 1)

In [20]:
start = default_timer()

results_2 = list(p.imap_unordered(extract_emb, tqdm.tqdm(batches)))

default_timer() - start

100%|██████████| 3/3 [00:00<00:00, 233.16it/s]


43.768461861000105

In [21]:
[l[0] for l in results_2]

['batch_1', 'batch_2', 'batch_0']

In [22]:
[l[0] for l in sorted(results_2)]

['batch_0', 'batch_1', 'batch_2']

In [23]:
arrays = [(b, np.concatenate(l)) for b, l in sorted(results_2)]
[a.shape for b, a in arrays]

[(102400,), (102400,), (91776,)]

In [24]:
for b, a in arrays:
    np.save('data/testing/embeddings/' + b + '.npy', a)